In [1]:
import pandas as pd
train_dataset = pd.read_csv('training-set.csv',delimiter=',')
judge_dataset = pd.read_csv('judge-set.csv',delimiter=',')


In [2]:
train_dataset.head()

,Tick#,offsetTime,IMU_ATTI(0):numSats,IMU_ATTI(0):barometer:Raw,IMU_ATTI(0):barometer:Smooth,IMU_ATTI(0):accel:X,IMU_ATTI(0):accel:Y,IMU_ATTI(0):accel:Z,IMU_ATTI(0):accel:Composite,IMU_ATTI(0):gyro:X,...,MotorCtrl:PWM:RFront,MotorCtrl:PWM:LFront,MotorCtrl:PWM:LBack,MotorCtrl:PWM:RBack,AirCraftCondition:fsmState,AirCraftCondition:nearGround,AirCraftCondition:landState,AirCraftCondition:UP_acc_t,AirCraftCondition:UP_TF_t,Operating_Phase
0,339062105,0.000,14,1188.3093,1188.0585,-0.007624,-0.162505,-0.980750,0.994151,-0.013848,...,NaN,NaN,NaN,NaN,0,True,1,0.0,0,Resting
1,339314363,0.066,14,1188.1595,1188.0468,-0.007572,-0.161164,-0.981163,0.994339,0.049079,...,NaN,NaN,NaN,NaN,0,True,1,0.0,0,Resting
2,339465911,0.105,14,1188.1595,1188.0468,-0.007572,-0.161164,-0.981163,0.994339,0.049079,...,NaN,NaN,NaN,NaN,0,True,1,0.0,0,Resting
3,339648466,0.152,14,1188.2122,1188.0359,-0.008646,-0.153160,-0.982913,0.994812,0.535372,...,NaN,NaN,NaN,NaN,1,True,1,0.0,0,Resting
4,339821470,0.197,14,1188.2122,1188.0359,-0.008646,-0.153160,-0.982913,0.994812,0.535372,...,NaN,NaN,NaN,NaN,1,True,1,0.0,0,Resting


In [3]:
total_number_of_rows = train_dataset.shape[0] 
total_number_of_columns = train_dataset.shape[1]
number_of_columns_non_numeric = len(train_dataset.select_dtypes(exclude=['number']).columns)
missing_values = train_dataset.isnull().sum()
missing_columns = [(column, missing_number) for column, missing_number in missing_values.items() if missing_number>0]
missing_columns.sort(key=lambda x: x[1], reverse=True)
summarizing =   f"total number of rows = {total_number_of_rows}\n"\
                f"total number of columns = {total_number_of_columns}\n"\
                f"number of columns having non-numeric values = {number_of_columns_non_numeric}\n"\
                f"columns with missing values = {missing_columns}\n"
print(summarizing)

total number of rows = 5085
total number of columns = 142
number of columns having non-numeric values = 5
columns with missing values = [('General:absoluteHeight', 5085), ('OA:frontDistance', 224), ('IMU_ATTI(1):directionOfTravel[true]', 32), ('IMU_ATTI(0):directionOfTravel[mag]', 8), ('IMU_ATTI(0):directionOfTravel[true]', 8), ('MotorCtrl:PWM:RFront', 5), ('MotorCtrl:PWM:LFront', 5), ('MotorCtrl:PWM:LBack', 5), ('MotorCtrl:PWM:RBack', 5), ('General:relativeHeight', 1)]



In [4]:
column_name = 'Operating_Phase'
X = train_dataset.drop(columns=[column_name])
y = train_dataset[column_name]

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=12346)
# Define the list of columns to extract
columns_to_drop = [
    'General:absoluteHeight',
    'GPS(0):Date',
    'GPS(0):Time',
    'GPS:dateTimeStamp',
    'IMUEX(0):err',
    'IMUEX(1):err',
    'IMUEX(0):rtk_Alti',
    'IMUEX(1):rtk_Alti'
]

X_train = X_train.drop(columns=columns_to_drop)
X_test = X_test.drop(columns=columns_to_drop)
tick_train, tick_test = list(X_train['Tick#']), list(X_test['Tick#'])
X_train = X_train.drop(columns='Tick#')
X_test = X_test.drop(columns='Tick#')


In [6]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder
categorical_columns = X_train.select_dtypes(exclude=['number']).columns
X_train_ohe = pd.get_dummies(X_train,  columns=categorical_columns, dtype=int)
X_test_ohe = pd.get_dummies(X_test,  columns=categorical_columns, dtype=int)
print(categorical_columns)

Index(['AirCraftCondition:nearGround'], dtype='object')


In [7]:
missing_values = X_train_ohe.isnull().sum()
missing_counts = pd.DataFrame({'variable_name': missing_values.index, 'num_of_missing_values': missing_values.values})
X_train_ohe_imputed = X_train_ohe.fillna(X_train_ohe.mean())
X_test_ohe_imputed = X_test_ohe.fillna(X_test_ohe.mean())

In [8]:

missing_values = X_train_ohe_imputed.isnull().sum()
missing_columns = [(column, missing_number) for column, missing_number in missing_values.items() if missing_number>0]
print(missing_columns, X_train_ohe_imputed.shape[1])

[] 133


In [9]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.fit_transform(y_test)
correlation_matrix = X_train_ohe_imputed.corrwith(pd.Series(y_train_encoded))

correlation_matrix.dropna(inplace=True)

top75_corr = correlation_matrix.abs()#.nlargest(70)
top75_df = pd.DataFrame({'variable': top75_corr.index, 'corr_score': top75_corr.values})
print(top75_df)


                               variable  corr_score
0                            offsetTime    0.014084
1                   IMU_ATTI(0):numSats    0.031180
2             IMU_ATTI(0):barometer:Raw    0.003898
3          IMU_ATTI(0):barometer:Smooth    0.002966
4                   IMU_ATTI(0):accel:X    0.025150
..                                  ...         ...
115          AirCraftCondition:fsmState    0.010033
116         AirCraftCondition:landState    0.011106
117          AirCraftCondition:UP_acc_t    0.033352
118  AirCraftCondition:nearGround_False    0.008650
119   AirCraftCondition:nearGround_True    0.008650

[120 rows x 2 columns]


C:\Users\Lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\Users\Lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


In [10]:
y_train_encoded

array([0, 1, 0, ..., 1, 0, 0])

In [11]:
top75_variables = top75_df['variable'].tolist()

X_train_t75 = X_train_ohe_imputed[top75_variables]
X_test_t75 = X_test_ohe_imputed[top75_variables]


In [12]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train_scaled_mm = scaler.fit_transform(X_train_t75)
X_test_scaled_mm = scaler.fit_transform(X_test_t75)

In [13]:
from sklearn.preprocessing import StandardScaler
scaler_s = StandardScaler()
X_train_scaled_std = scaler_s.fit_transform(X_train_t75)
X_test_scaled_std = scaler_s.fit_transform(X_test_t75)
print(X_train_scaled_std,X_test_scaled_std)

[[ 0.50026704  0.05671208 -0.17571346 ... -0.55489646  0.50245702
  -0.50245702]
 [-0.58506224  1.53558867  1.07014539 ... -0.55489646  0.50245702
  -0.50245702]
 [-0.13530272  1.53558867  0.00977652 ...  1.96312108  0.50245702
  -0.50245702]
 ...
 [-0.52204196  1.53558867  0.84491022 ... -0.55489646  0.50245702
  -0.50245702]
 [ 1.51624083  0.05671208 -0.90714055 ... -0.55489646  0.50245702
  -0.50245702]
 [-1.18504131  0.05671208  2.35095839 ... -0.55489646  0.50245702
  -0.50245702]] [[ 1.09079930e-03 -2.30397622e-02  3.64907593e-02 ... -5.61968810e-01
   4.77766543e-01 -4.77766543e-01]
 [ 1.72054829e+00 -1.58513564e+00 -1.25013216e+00 ... -5.61968810e-01
  -2.09307247e+00  2.09307247e+00]
 [-1.11968646e-01 -2.30397622e-02  5.91591867e-02 ... -5.61968810e-01
   4.77766543e-01 -4.77766543e-01]
 ...
 [-1.38512919e+00 -2.30397622e-02 -8.90102025e-01 ... -5.61968810e-01
  -2.09307247e+00  2.09307247e+00]
 [-4.54434906e-01  1.53905612e+00 -4.03720847e-01 ... -5.61968810e-01
   4.77766543

In [14]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Flatten, BatchNormalization, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.regularizers import l2

num_classes = 5
model = Sequential()

model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train_t75.shape[1], 1)))
# model.add(BatchNormalization())  # Add batch normalization here
model.add(MaxPooling1D(pool_size=2))

model.add(LSTM(units=59, return_sequences=True))
model.add(Dropout(0.5))


model.add(Flatten())
model.add(Dense(units=num_classes, activation='softmax', kernel_regularizer=l2(0.01)))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

num_classes = 5 
y_train_encoded = to_categorical(y_train_encoded, num_classes=num_classes)

model.fit(X_train_t75, y_train_encoded, epochs=8, batch_size=32)

y_test_encoded = to_categorical(y_test_encoded, num_classes=num_classes)
test_loss, test_accuracy = model.evaluate(X_test_t75, y_test_encoded)

print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


Epoch 1/8
128/128 [==============================] - 8s 39ms/step - loss: 0.9391 - accuracy: 0.5927
Epoch 2/8
128/128 [==============================] - 5s 39ms/step - loss: 0.6589 - accuracy: 0.7507
Epoch 3/8
128/128 [==============================] - 5s 39ms/step - loss: 0.5368 - accuracy: 0.8065
Epoch 4/8
128/128 [==============================] - 5s 38ms/step - loss: 0.4830 - accuracy: 0.8346
Epoch 5/8
128/128 [==============================] - 5s 39ms/step - loss: 0.4356 - accuracy: 0.8594
Epoch 6/8
128/128 [==============================] - 5s 41ms/step - loss: 0.4433 - accuracy: 0.8387
Epoch 7/8
128/128 [==============================] - 5s 41ms/step - loss: 0.4121 - accuracy: 0.8547
Epoch 8/8
32/32 [==============================] - 1s 19ms/step - loss: 0.3147 - accuracy: 0.9095
Test Accuracy: 90.95%


In [15]:
judge_data = judge_dataset.drop(columns=columns_to_drop)

In [16]:
categorical_columns = judge_data.select_dtypes(exclude=['number']).columns
judge_data_ohe = pd.get_dummies(judge_data,  columns=categorical_columns, dtype=int)
missing_values = judge_data_ohe.isnull().sum()
missing_counts = pd.DataFrame({'variable_name': missing_values.index, 'num_of_missing_values': missing_values.values})
judge_data_ohe_imputed = judge_data_ohe.fillna(judge_data_ohe.mean())
print(judge_data_ohe.columns)
judge_data_ohe['AirCraftCondition:nearGround_False'] = 0
print(judge_data_ohe.head())

Index(['Tick#', 'offsetTime', 'IMU_ATTI(0):numSats',
       'IMU_ATTI(0):barometer:Raw', 'IMU_ATTI(0):barometer:Smooth',
       'IMU_ATTI(0):accel:X', 'IMU_ATTI(0):accel:Y', 'IMU_ATTI(0):accel:Z',
       'IMU_ATTI(0):accel:Composite', 'IMU_ATTI(0):gyro:X',
       ...
       'IMU_ATTI(1):temperature', 'MotorCtrl:PWM:RFront',
       'MotorCtrl:PWM:LFront', 'MotorCtrl:PWM:LBack', 'MotorCtrl:PWM:RBack',
       'AirCraftCondition:fsmState', 'AirCraftCondition:landState',
       'AirCraftCondition:UP_acc_t', 'AirCraftCondition:UP_TF_t',
       'AirCraftCondition:nearGround_True'],
      dtype='object', length=133)
       Tick#  offsetTime  IMU_ATTI(0):numSats  IMU_ATTI(0):barometer:Raw  \
0  143222258       0.000                   11                   1230.502   
1  143393138       0.044                   11                   1230.502   
2  143566452       0.089                   11                   1230.404   
3  143758752       0.139                   11                   1230.404   
4  1

In [17]:
judge_data_t75 = judge_data_ohe[top75_variables]
X_judge_scaled_std = scaler_s.fit_transform(judge_data_t75)
 

In [146]:
import numpy as np
predicted_probabilities = model.predict(X_judge_scaled_std) 
class_labels = ['Resting', 'Taking-Off', 'Hovering', 'Climbing', 'Landing']
predicted_operating_phase_labels = [class_labels[np.argmax(prediction)] for prediction in predicted_probabilities]
new_dataset = judge_data[["Tick#"]].copy()
new_dataset["Operating_Phase"] = predicted_operating_phase_labels
new_dataset.to_csv("3.csv", index=False)

207/207 [==============================] - 2s 7ms/step
